# TODO

* Add actions:
  - Delete/edit feedback option
  - Switch students
* Hook up database
* Make a dashboard layout
* Deploy

In [55]:
from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image
import pandas
import jinja2
import time

In [56]:
results = pandas.read_excel('Results TN2844 2016-2.xls')

students = list(map(lambda numbername: "{0} ({1})".format(*numbername), zip(results.Number, results.Name)))

In [59]:
autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {% for option in options %}"{{ option }}",
    {% endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

selector = "placeholder='# or name'"

def add_option(target):
    def wrapper(sender):
        with target.hold_trait_notifications():
            target.options = target.options + [sender.value]
        sender.value = ''
    return wrapper

# All the individual widgets

prev_stud = Button(icon='fa-fast-backward', description=' previous')
next_stud = Button(icon='fa-fast-forward', description=' submit & next', button_style='warning')
next_stud_in_list = Button(icon='fa-forward', description=' next')
jump_to_stud = Text(description='Jump to', placeholder='# or name')

previous_problem = Button(icon='fa-arrow-circle-up', description=' previous problem')
next_problem = Button(icon='fa-arrow-circle-down', description=' next problem')

options_feedback = SelectMultiple(name='feedback', options=['Everything correct', 'No solution provided'])
new_option = Text(placeholder='New feedback option')
new_option.on_submit(add_option(options_feedback))
student_specific = Textarea(placeholder='Student-specific feedback')
with open('test_exam/test_page_00.png', 'rb') as f:
    data = f.read()
answer = Image(value=data, format='png', width='100%')

widgets = VBox([
        HBox([prev_stud, next_stud, next_stud_in_list, jump_to_stud]),
        HBox([next_problem, previous_problem]),
        HBox([VBox([options_feedback, new_option]), student_specific]),
        answer
])

display.display(widgets)
time.sleep(.1)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=students)))
display.display_javascript(display.Javascript(prevent_unselect))

In [47]:
answer

In [53]:
from PIL import Image
import zbarlight

file_path = 'qr_code_businesscard.jpg'
with open(file_path, 'rb') as image_file:
    image = Image.open(image_file)
    image.load()

codes = zbarlight.scan_codes('qrcode', image)
print('QR codes: %s' % codes)

QR codes: [b'BEGIN:VCARD\nN:Sergio Garc\xef\xbe\x83\xef\xbd\xada Caso\nTEL:902286386\nEMAIL:sergio.garcia@treelogic.com\nEND:VCARD']
